In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
# Read the spotify data in from data.csv
url = 'https://media.githubusercontent.com/media/jossharlequin/spotify-popularity-project/main/Resources/sql_spotify_data.csv'
spotify_df = pd.read_csv(url)
spotify_df.head()

,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,duration_seconds
0,0.995,0.708,0.1950,0,0.563,10,0.1510,-12.428,1,0,0.0506,118.469,0.7790,158.648
1,0.994,0.379,0.0135,0,0.901,8,0.0763,-28.454,1,0,0.0462,83.972,0.0767,282.133
2,0.604,0.749,0.2200,0,0.000,5,0.1190,-19.924,0,0,0.9290,107.177,0.8800,104.300
3,0.995,0.781,0.1300,0,0.887,1,0.1110,-14.734,0,0,0.0926,108.003,0.7200,180.760
4,0.990,0.210,0.2040,0,0.908,11,0.0980,-16.829,1,1,0.0424,62.149,0.0693,687.733


In [3]:
# Setting popularity as the target variable and setting the remaining columns as features
y = spotify_df.popularity.values
X = spotify_df.drop(columns='popularity').values

In [41]:
# Scaling the data using StandarScaler as a preprocessing step for the neural network
# Creating the StandardScalar instance
scaler = StandardScaler()

# Fitting the X data
X_scaler = scaler.fit(X)

# Scaling the X data
X_scaled = X_scaler.transform(X)

# Splitting training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

In [42]:
# Binning the data based on data range 
bins = [-1, 50, 100]
labels = [0, 1]
y_train_binned = pd.cut(y_train, bins=bins, labels=labels)
y_test_binned = pd.cut(y_test, bins=bins, labels=labels)

# Encoding binned data into categorical data 
encoder = OneHotEncoder(sparse=False)
y_train_one_hot = encoder.fit_transform(y_train_binned.reshape(-1,1))
y_test_one_hot = encoder.transform(y_test_binned.reshape(-1,1))

c:\Users\josep\anaconda3\envs\dev\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [43]:
# Defining the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=13))
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu"))
nn_model.add(tf.keras.layers.Dense(2, activation="softmax"))

# Compile the sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
fit_model = nn_model.fit(X_train, y_train_one_hot, epochs=100)

# Evaluating the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Finding predicted y classes
y_predicted = nn_model.predict(X_test)

# Converting y_predicted and y_test_one_hot into integer values for confusion matrix and accuracy score
y_test_classes = np.argmax(y_test_one_hot, axis=1)
y_predicted_classes = np.argmax(y_predicted, axis=1)

# Creating confusion matrix
cm = confusion_matrix(y_test_classes, y_predicted_classes)
cm_df = pd.DataFrame(
    cm, index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1']
)

# Creating accuracy score
acc_score = accuracy_score(y_test_classes, y_predicted_classes)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score: {acc_score}")
print("Classification Report")
print(classification_report(y_test_classes, y_predicted_classes))

Epoch 1/100
3983/3983 [==============================] - 10s 2ms/step - loss: 0.4002 - accuracy: 0.8188
Epoch 2/100
3983/3983 [==============================] - 9s 2ms/step - loss: 0.3831 - accuracy: 0.8261
Epoch 3/100
3983/3983 [==============================] - 8s 2ms/step - loss: 0.3786 - accuracy: 0.8288
Epoch 4/100
3983/3983 [==============================] - 8s 2ms/step - loss: 0.3766 - accuracy: 0.8292
Epoch 5/100
3983/3983 [==============================] - 9s 2ms/step - loss: 0.3755 - accuracy: 0.8306
Epoch 6/100
3983/3983 [==============================] - 9s 2ms/step - loss: 0.3748 - accuracy: 0.8305
Epoch 7/100
3983/3983 [==============================] - 9s 2ms/step - loss: 0.3740 - accuracy: 0.8310
Epoch 8/100
3983/3983 [==============================] - 8s 2ms/step - loss: 0.3733 - accuracy: 0.8313
Epoch 9/100
3983/3983 [==============================] - 9s 2ms/step - loss: 0.3730 - accuracy: 0.8314
Epoch 10/100
3983/3983 [==============================] - 9s 2ms/step - 

,Predicted 0,Predicted 1
Actual 0,31659,1809
Actual 1,5192,3818


Accuracy Score: 0.8351852723762889
Classification Report
              precision    recall  f1-score   support

           0       0.86      0.95      0.90     33468
           1       0.68      0.42      0.52      9010

    accuracy                           0.84     42478
   macro avg       0.77      0.68      0.71     42478
weighted avg       0.82      0.84      0.82     42478



In [4]:
# Binning the data into equal sized groups
labels=[0,1]
q_binned_data = pd.qcut(spotify_df['popularity'], 2, labels=labels, precision=0)

# Scaling the data using StandarScaler as a preprocessing step for the neural network
# Creating the StandardScalar instance
scaler = StandardScaler()

# Fitting the X data
X_scaler = scaler.fit(X)

# Scaling the X data
X_scaled = X_scaler.transform(X)

# Splitting training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, q_binned_data, random_state=42)

# Convert y_train and y_test from Pandas Series to NumPy arrays
y_train_np = np.array(y_train).reshape(-1, 1)
y_test_np = np.array(y_test).reshape(-1, 1)

# Encoding binned data into categorical data
encoder = OneHotEncoder(sparse=False)
y_train_one_hot = encoder.fit_transform(y_train_np)
y_test_one_hot = encoder.transform(y_test_np)

pandas.core.series.Series

c:\Users\josep\anaconda3\envs\dev\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
# Defining the neural network
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=13))
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu"))
nn_model.add(tf.keras.layers.Dense(2, activation="softmax"))

# Compile the sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
fit_model = nn_model.fit(X_train, y_train_one_hot, epochs=50)

# Evaluating the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Finding predicted y classes
y_predicted = nn_model.predict(X_test)

# Converting y_predicted and y_test_one_hot into integer values for confusion matrix and accuracy score
y_test_classes = np.argmax(y_test_one_hot, axis=1)
y_predicted_classes = np.argmax(y_predicted, axis=1)

# Creating confusion matrix
cm = confusion_matrix(y_test_classes, y_predicted_classes)
cm_df = pd.DataFrame(
    cm, index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1']
)

# Creating accuracy score
acc_score = accuracy_score(y_test_classes, y_predicted_classes)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score: {acc_score}")
print("Classification Report")
print(classification_report(y_test_classes, y_predicted_classes))



Epoch 1/50


3983/3983 [==============================] - 13s 3ms/step - loss: 0.4746 - accuracy: 0.7682
Epoch 2/50
3983/3983 [==============================] - 11s 3ms/step - loss: 0.4513 - accuracy: 0.7813
Epoch 3/50
3983/3983 [==============================] - 11s 3ms/step - loss: 0.4475 - accuracy: 0.7816
Epoch 4/50
3983/3983 [==============================] - 12s 3ms/step - loss: 0.4454 - accuracy: 0.7833
Epoch 5/50
3983/3983 [==============================] - 13s 3ms/step - loss: 0.4441 - accuracy: 0.7835
Epoch 6/50
3983/3983 [==============================] - 12s 3ms/step - loss: 0.4430 - accuracy: 0.7850
Epoch 7/50
3983/3983 [==============================] - 11s 3ms/step - loss: 0.4419 - accuracy: 0.7847
Epoch 8/50
3983/3983 [==============================] - 12s 3ms/step - loss: 0.4410 - accuracy: 0.7853
Epoch 9/50
3983/3983 [==============================] - 11s 3ms/step - loss: 0.4405 - accuracy: 0.7862
Epoch 10/50
3983/3983 [==============================] - 11s 3ms/step

,Predicted 0,Predicted 1
Actual 0,16957,4151
Actual 1,4879,16491


Accuracy Score: 0.7874193700268374
Classification Report
              precision    recall  f1-score   support

           0       0.78      0.80      0.79     21108
           1       0.80      0.77      0.79     21370

    accuracy                           0.79     42478
   macro avg       0.79      0.79      0.79     42478
weighted avg       0.79      0.79      0.79     42478

